# Hybrid / Compress

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# 출력 포메팅
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, max_len=140, title="Retriever Results"):
    table = Table(title=title)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        table.add_row(str(i), src, page, (text[:max_len] + ("…" if len(text) > max_len else "")))

    console.print(table)

## 1. db 생성 및 로드

In [4]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(model='text-embedding-3-small')
persist_directory = '../vectorstore/samsung_2025_re'
collection_name = 'samsung_2025'

c:\walker\code\langchain-basic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_community.document_loaders import PyPDFLoader

docs = PyPDFLoader("../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf").load()
len(docs)

87

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

chunks = splitter.split_documents(docs)
len(chunks)

237

In [7]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    collection_name=collection_name,
    persist_directory=persist_directory,
    embedding=embedding
)

vectorstore

In [8]:
load_vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
    collection_name=collection_name
)

load_vectorstore

## 2. Ensemble Retriever(Dense + Parse)

In [9]:
# Dense Retriever
sim_retriever = load_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5
    }
)

In [10]:
db_docs = load_vectorstore._collection.get(include=['documents', 'metadatas'])
db_docs['metadatas'][:2]

[{'creator': 'Adobe InDesign 15.1 (Macintosh)',
  'creationdate': '2025-07-10T16:11:16+09:00',
  'page_label': '1',
  'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'producer': 'Adobe PDF Library 15.0',
  'total_pages': 87,
  'trapped': '/False',
  'page': 0,
  'moddate': '2025-09-04T16:51:11+09:00'},
 {'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'total_pages': 87,
  'trapped': '/False',
  'page': 1,
  'page_label': '2',
  'creationdate': '2025-07-10T16:11:16+09:00',
  'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 15.1 (Macintosh)',
  'moddate': '2025-09-04T16:51:11+09:00'}]

In [11]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.documents import Document

bm_docs = []

# chunk된 db의 내용을 가져와서 사용 => chunk 보다는 영속된 db의 내용을 쓰는 것이 안정적
for content, meta in zip(db_docs['documents'], db_docs['metadatas']):
    bm_docs.append(Document(page_content = content, metadata = meta))
    
bm_docs[:3]

[Document(metadata={'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'page_label': '1', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'producer': 'Adobe PDF Library 15.0', 'total_pages': 87, 'trapped': '/False', 'page': 0, 'moddate': '2025-09-04T16:51:11+09:00'}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'),
 Document(metadata={'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'trapped': '/False', 'page': 1, 'page_label': '2', 'creationdate': '2025-07-10T16:11:16+09:00', 'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'moddate': '2025-09-04T16:51:11+09:00'}, page_content='삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable Future\

In [12]:
# Parse Retriever
bm25_retriever = BM25Retriever.from_documents(bm_docs)
bm25_retriever.k = 5

In [13]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[sim_retriever, bm25_retriever],
    weights=[0.7, 0.3]
)

In [14]:
question = '삼성전자의 2025년 전망은?'
result = hybrid_retriever.invoke(question)
rich_docs(result, title='hybrid result')

                                                   hybrid result                                                   
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey        │
│   │                                                  │      │ Towards   a Sustainable Future A Journey  Towards │
│   │                                                  │      │ a Sustainable Future                              │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,       │
│   │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성장  │
│   │                                                  │      │ …                                                 │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │   77 │ 삼성전자 지속가능경영보고서 2025 77 Scope 1, 2    │
│   │                                                  │      │ 온실가스 배출량 검증 의견서 Our Company           │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86               │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영    │
│   │                                                  │      │ 보고 기준인 GRI(G…                                │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │    6 │ 삼성전자 지속가능경영보고서 2025 06 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주요 분야별 협의회  환경     [DX부문] 환경안전    │
│   │                                                  │      │ 회의, ESG공시TF  [DS부문]  탄소감축위원…          │
│ 6 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트         │
│   │                                                  │      │ http://www.samsung.com/sec/sustainability/main ·  │
│   │                                                  │      │ 삼성전자주식회사 IR 웹사이트                      │
│   │                                                  │      │ http://www.samsung.com/sec/ir ·  삼성전자주식회 … │
│   │                                                  │      │ 뉴…                                               │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

## 3. 압축 Retriever
- 검색 결과가 내용이 너무 길때
- 검색 결과가 파편화(띄어쓰기/문단구분) 되어 있는 경우
- => 비용 문제는 고려할 것
- => 전처리 단계에서 해결하거나 로컬 요약을 사용하는 것도 고려

In [15]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter
from langchain_openai import ChatOpenAI

compressor = LLMChainExtractor.from_llm(ChatOpenAI(
    model_name='gpt-4.1-mini',
    temperature=0
    )
)

com_retriever = ContextualCompressionRetriever(
    base_retriever=sim_retriever,
    base_compressor=compressor
)

In [16]:
question = '삼성 전자의 목표와 기준 년도만 간단히 알려줘'

com_result = com_retriever.invoke(question)
rich_docs(com_result, title='compress result')

                                                  compress result                                                  
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자는 2022년 9월 발표한 '新환경경영전략'을   │
│   │                                                  │      │ 기반으로 탄소중립 달성, 자원순환 극대화, 그리고   │
│   │                                                  │      │ 기술 혁신을 통한 환경 난제 해결을 위해 노력하고   │
│   │                                                  │      │ 있습니다.   DX(Device eXperience)부문은 2030년    │
│   │                                                  │      │ 탄소중립 달성을 목표로   …                        │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자는 2022년 9월 발표한 '新환경경영전략'을   │
│   │                                                  │      │ 기반으로 탄소중립 달성, 자원순환 극대화, 그리고   │
│   │                                                  │      │ 기술 혁신을 통한 환경 난제 해결을 위해 노력하고   │
│   │                                                  │      │ 있습니다.   DX(Device eXperience)부문은 2030년    │
│   │                                                  │      │ 탄소중립 달성을 목표로   …                        │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025                  │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025                  │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025                  │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다.   보고 기간   2024년 1월1일부터       │
│   │                                                  │      │ 2024년 12월 …                                     │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

## 3-1. 압축 Retriever(Embedding - No Cost : 잘 안씀)

In [22]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

# 필터만 거치는거라 비용은 들지 않음
compressor = EmbeddingsFilter(
    embeddings=embedding,
    similarity_threshold=0.2
)

# 중복이 많은 경우 활용
mmr_retriever = load_vectorstore.as_retriever(
    search_type = 'mmr',
    search_kwargs = {'k' : 5,
                    'fetch_k' : 10,
                    'lambda_mult' : 0.5 # 1 : 관련성, 0 : 다양성
                    }
)

comp_embed = ContextualCompressionRetriever(
    base_retriever=mmr_retriever, 
    base_compressor=compressor
)

comp_embed_result = comp_embed.invoke(question)
rich_docs(comp_embed_result)

                                                 Retriever Results                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,       │
│   │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성장  │
│   │                                                  │      │ …                                                 │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey        │
│   │                                                  │      │ Towards   a Sustainable Future A Journey  Towards │
│   │                                                  │      │ a Sustainable Future                              │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86               │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영    │
│   │                                                  │      │ 보고 기준인 GRI(G…                                │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   84 │ 삼성전자 지속가능경영보고서 2025 84 코드 공시     │
│   │                                                  │      │ 항목 참고 페이지 및 답변 제품 안전 TC-HW-230a.1   │
│   │                                                  │      │ 제품의 정보보안 관련 리스크를 발견하고 이를       │
│   │                                                  │      │ 개선하는 방법 설명 삼성전자는 개인정보보호팀장,   │
│   │                                                  │      │ 정보보호센터장이 개인정보보호 운영위원회와 보안…  │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   78 │ 삼성전자 지속가능경영보고서 2025 78 Scope 3       │
│   │                                                  │      │ 온실가스 배출량 검증 의견서 Our Company           │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘